In [69]:
import pandas as pd
from glob import glob
import wave
from tqdm import tqdm
import numpy as np
import nlptutti as metrics

In [59]:
file_list=glob('predict/english/*')

### duration 구하기
- 음성파일 길이 정보

In [10]:
from eng_STT_API import get_audioPath

In [55]:
predict_path='predict/english/'
data_path='data/'

dataset=input("데이터 선택 입력해주세요: (1, 2, 3, 4 택1) ")
if dataset=="1":
    predict=predict_path+'1_'
    loaded_list=list(pd.read_csv(data_path+"common voice/sample/sample.csv")['filename'])
elif dataset=="2":
    with open(data_path+"2_VCTK-Corpus/sample.txt", 'r') as file:
        # 파일의 각 줄을 읽어와 리스트에 저장
        loaded_list = [line.strip() for line in file.readlines()]
    predict=predict_path+'2_'
elif dataset=="3":
    predict=predict_path+'3_'
    loaded_list=list(pd.read_csv(data_path+"3_CMU/sample.csv")["file_path"])
elif dataset=="4":
    predict=predict_path+'4_'
    loaded_list=list(pd.read_csv(data_path+"4_IEMOCAP/sample.csv")["titre"])
else:
    print("틀렸습니다. 이 파일을 재실행해주세요")
    
duration_list=[]
for file in tqdm(loaded_list):
    audio_file=get_audioPath(file, dataset)
    audio = wave.open(audio_file)
    frames = audio.getnframes()
    rate = audio.getframerate()
    duration = frames / float(rate)
    duration_list.append([file, duration])
df=pd.DataFrame(duration_list)
df.columns=['file_name', 'duration']
df.to_csv('cerwer/english/'+dataset+'_duration.csv', index=False)

데이터 선택 입력해주세요: (1, 2, 3, 4 택1) 4


100%|███████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 18105.59it/s]


### 전처리
- 소문자로 바꾸기
- cer, wer 계산

In [71]:
def get_cerwer(df):
    df['cer']=0
    df['wer']=0
    for i in range(len(df)):
        refs=df.iloc[i, 1]
        preds=df.iloc[i, 2]
        result = metrics.get_cer(refs, preds)
        cer = result['cer']
        result_wer=metrics.get_wer(refs, preds)
        wer=result_wer['wer']
        df.iloc[i, 4]=cer
        df.iloc[i, 5]=wer
    return df

In [73]:
for file in file_list:    
    file_name=file.split("english\\")[-1]
    save_name='cerwer/english/cw_'+file_name
    df=pd.read_csv(file)
    
    # 아예 인식 못한 데이터 분리
    null_df=df[df['recognized_text'].isnull()]
    df=df[~df['recognized_text'].isnull()]
    
    # 소문자로 바꾸기
    df['answer_text']=df['answer_text'].apply(lambda x:str(x).lower())
    df['recognized_text']=df['recognized_text'].apply(lambda x:str(x).lower())
    total=get_cerwer(df)
    
    null_df['cer']=np.nan
    null_df['wer']=np.nan
    
    total=pd.concat([total, null_df])
    total.reset_index(inplace=True, drop=True)

    total.to_csv(save_name, index=False)

C:\Users\82105\AppData\Local\Temp\ipykernel_14960\2583305088.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.6153846153846154' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.iloc[i, 4]=cer
C:\Users\82105\AppData\Local\Temp\ipykernel_14960\2583305088.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.iloc[i, 5]=wer
C:\Users\82105\AppData\Local\Temp\ipykernel_14960\2583305088.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.47619047619047616' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.iloc[i, 4]=cer
C:\Users\82105\AppData\Local\Temp\ipykernel_14960\2583305088.py:12: FutureW

C:\Users\82105\AppData\Local\Temp\ipykernel_14960\2583305088.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.11764705882352941' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.iloc[i, 4]=cer
C:\Users\82105\AppData\Local\Temp\ipykernel_14960\2583305088.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.25' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.iloc[i, 5]=wer
C:\Users\82105\AppData\Local\Temp\ipykernel_14960\2583305088.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.11764705882352941' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.iloc[i, 4]=cer
C:\Users\82105\AppData\Local\Temp\ipykernel_14960\2583305088.py:12: Futur

### 분석

In [74]:
file_list=glob('cerwer/english/*')

In [79]:
# 음원 파일의 길이가 담긴 데이터프레임 불러오기
duration=[]
for file in file_list:
    if "duration" in file:
        duration.append(file)

duration_df=pd.DataFrame()
for d in duration:
    tmp=pd.read_csv(d)
    duration_df=pd.concat([duration_df, tmp])
duration_df.reset_index(inplace=True, drop=True)

In [80]:
def get_corr(df):
    correlation = df['execution_time'].corr(df['duration'])
    return correlation

In [86]:
file_list[1].split('english\\')[-1].split('_')[0]

'2'

In [94]:
def analyze(duration_df):
    api=input("naver, azure, google, whisper 중 하나 입력: ")
    target=[]
    for file in file_list:
        if api in file:
            target.append(file)
    zhengli=[]
    for t in target:
        dataset=t.split('english\\')[-1].split('_')[1]        
        df=pd.read_csv(t)
        
        # 인식 안된 데이터 제외 및 카운트
        null_df=df[((df['recognized_text'].isnull())|(df['recognized_text']=='error'))]
        df=df[~((df['recognized_text'].isnull())|(df['recognized_text']=='error'))]
        
        # 문자열일경우
        df['execution_time']=df['execution_time'].astype('float64')
        
        null_cnt=len(null_df)
        df=pd.merge(df, duration_df, on='file_name', how='left')
        duration_exetime_corr=get_corr(df)
        at_average_length = df['answer_text'].str.len().mean()
        rt_average_length = df['recognized_text'].str.len().mean()
        time_average=df['execution_time'].mean()
        cer_average=df['cer'].mean()
        wer_average=df['wer'].mean()
        duration_average=df['duration'].mean()
        zhengli.append([api, dataset,null_cnt, at_average_length, rt_average_length, time_average,cer_average,wer_average, duration_average, duration_exetime_corr])
    zhengli=pd.DataFrame(zhengli)
    zhengli.columns=['api','dataset','null_cnt', 'at_average_length', 'rt_average_length', 'time_average','cer_average','wer_average', 'duration_average', 'duration_exetime_corr']
    return zhengli        
        

In [96]:
naver=analyze(duration_df)

naver, azure, google, whisper 중 하나 입력: naver


In [97]:
naver

,api,dataset,null_cnt,mt_average_length,rt_average_length,time_average,cer_average,wer_average,duration_average,duration_exetime_corr
0,naver,1,0,48.44200,48.105000,1.404001,0.099334,0.166582,4.402584,0.780430
1,naver,2,0,40.58800,38.987000,1.051870,0.038519,0.074687,3.617811,0.865596
2,naver,3,0,48.52100,47.421000,1.119858,0.063919,0.126452,3.213521,0.777399
3,naver,4,20,62.07449,57.670408,1.664970,0.201276,0.302548,4.673295,0.882511


In [98]:
azure=analyze(duration_df)

naver, azure, google, whisper 중 하나 입력: azure


In [99]:
azure

,api,dataset,null_cnt,mt_average_length,rt_average_length,time_average,cer_average,wer_average,duration_average,duration_exetime_corr
0,azure,1,3,48.408225,49.238716,1.044604,0.015319,0.021817,4.382877,0.276748
1,azure,2,2,40.546092,40.327655,0.823382,0.011638,0.028551,3.608685,0.907783
2,azure,3,0,48.521000,48.200000,0.816520,0.013245,0.026874,3.213521,0.877197
3,azure,4,34,62.375776,51.828157,1.079487,0.160681,0.198561,4.674635,0.482894


In [100]:
google=analyze(duration_df)

naver, azure, google, whisper 중 하나 입력: google


In [101]:
google

,api,dataset,null_cnt,mt_average_length,rt_average_length,time_average,cer_average,wer_average,duration_average,duration_exetime_corr
0,google,1,13,48.514691,46.021277,1.219863,0.119198,0.182337,4.406128,0.666082
1,google,2,0,40.588000,38.768000,0.995530,0.064585,0.121471,3.617811,0.769331
2,google,3,0,48.521000,46.447000,0.908533,0.062996,0.130727,3.213521,0.632118
3,google,4,54,63.593023,48.165962,1.128894,0.214372,0.271115,4.745645,0.934393


In [102]:
whisper=analyze(duration_df)

naver, azure, google, whisper 중 하나 입력: whisper


In [103]:
whisper

,api,dataset,null_cnt,mt_average_length,rt_average_length,time_average,cer_average,wer_average,duration_average,duration_exetime_corr
0,whisper,1,0,48.442000,49.672000,1.576066,0.016939,0.033371,4.402584,0.054028
1,whisper,2,0,40.588000,40.432000,1.669407,0.010281,0.024330,3.617811,0.107989
2,whisper,3,1,48.494494,48.221221,1.334205,0.015061,0.033812,3.212368,-0.028763
3,whisper,4,2,61.101202,60.547094,1.621367,0.088184,0.128358,4.626556,0.094838


In [107]:
def total_analyze(duration_df):
    total=pd.DataFrame()
    api_list=['naver','azure','google','whisper']
    for api in api_list:
        target=[]
        for file in file_list:
            if api in file:
                target.append(file)

        zhengli=[]
        for t in target:
            dataset=t.split('cw_')[1].split('_'+api)[0]
            df=pd.read_csv(t)

            # 인식 안된 데이터 제외 및 카운트
            null_df=df[((df['recognized_text'].isnull())|(df['recognized_text']=='error'))]
            df=df[~((df['recognized_text'].isnull())|(df['recognized_text']=='error'))]

            # 문자열일경우
            df['execution_time']=df['execution_time'].astype('float64')

            null_cnt=len(null_df)
            df=pd.merge(df, duration_df, on='file_name', how='left')
            duration_exetime_corr=get_corr(df)
            at_average_length = df['answer_text'].str.len().mean()
            rt_average_length = df['recognized_text'].str.len().mean()
            time_average=df['execution_time'].mean()
            cer_average=df['cer'].mean()
            wer_average=df['wer'].mean()
            duration_average=df['duration'].mean()
            zhengli.append([api, dataset,null_cnt, at_average_length, rt_average_length, time_average,cer_average,wer_average, duration_average,duration_exetime_corr])
        zhengli=pd.DataFrame(zhengli)
        zhengli.columns=['api','dataset','null_cnt', 'at_average_length', 'rt_average_length', 'time_average','cer_average','wer_average', 'duration_average', 'duration_exetime_corr']
        zhengli=zhengli.groupby(['api'])[['null_cnt','time_average','cer_average','wer_average','duration_exetime_corr']].mean()
        zhengli.reset_index(inplace=True)
        total=pd.concat([total, zhengli])
    return total

In [108]:
total=total_analyze(duration_df)

In [109]:
total

,api,null_cnt,time_average,cer_average,wer_average,duration_exetime_corr
0,naver,5.00,1.310175,0.100762,0.167567,0.826484
0,azure,9.75,0.940998,0.050221,0.068951,0.636156
0,google,16.75,1.063205,0.115288,0.176413,0.750481
0,whisper,0.75,1.550261,0.032616,0.054968,0.057023
